In [3]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates

import ind.alpha_base10 as base10


%load_ext autoreload
%autoreload 2
%aimport ind.alpha_base10



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#读取已经缓存的复权数据，并预处理
stock_df = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
smpl.optimize_data_type(stock_df)
## 日无风险回报
ret_fs_data = pd.read_csv(module_path+'/data/static/china10yearbond.csv').set_index('date').sort_index()
ret_fs_data = (ret_fs_data['high'].astype(np.float32)+ret_fs_data['low'].astype(np.float32))/2 * 0.01
ret_fs_daily = ret_fs_data/252

ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)


#############测试#######################
# ['000001','000008','600600','000729']
# stock_df = stock_df_full.loc[(slice(None),stock_df_full.index.get_level_values(1).unique().to_list()[0:22]),:]
# ret_t, ret_t_excess, market_value_t, ret_excess_market_t = base10.prepare_data(stock_df,ret_fs_daily)

stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)


def generate_alpha_factors(codes, data_df=None, ret_fs=None, ret=None, ret_excess=None, market_value=None, ret_excess_market=None):
    import ind.alpha_base10 as base10
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd

    df = data_df.loc[(slice(None),codes),:]
    ret_ = ret.loc[(slice(None),codes)]
    ret_excess_ = ret_excess.loc[(slice(None),codes)]
    

    alpha_beta = base10.camp_beta_alpha(ret_excess_,ret_excess_market)
    mom = base10.momentum(ret_,ret_fs)
    size_lg = base10.sizelg(df)
    bp = base10.bp(df)
    
    return pd.concat([alpha_beta, mom, size_lg, bp], axis=1).loc[(slice(None),codes),:]


worker=6
task = task_chunk_split(stock_codes, worker)
results = parallal_task(worker, generate_alpha_factors, task, 
                        data_df=stock_df, ret_fs=ret_fs_daily, ret=ret_t, 
                        ret_excess=ret_t_excess, market_value=market_value_t,
                        ret_excess_market=ret_excess_market_t)
results = pd.concat(results).sort_index()

####使用全局数据的项目，注意不要分步计算
earnings_yield = base10.earnings_yield(ret_t,market_value_t,stock_df['industry']) 
print('earnings_yield')
liquidity = base10.liquidity(stock_df)
print('liquidity')
resvol = base10.resvol(ret_t, ret_fs_daily, ret_t_excess, results['sizelg'], results['beta'], results['residual'])
print('resvol')
sizenl = base10.sizenl(results['sizelg'])

final = pd.concat([results, earnings_yield, liquidity, resvol, sizenl],axis=1).sort_index()
save_cache('base10_train', final, cache_type=CACHE_TYPE.FACTOR)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:22324  pid:3432


  0%|          | 0/6 [00:00<?, ?it/s]

earnings_yield
liquidity
resvol


In [5]:
load_cache('base10_train',cache_type=CACHE_TYPE.FACTOR).dropna()

beta     alpha  residual  momentum     sizelg  \
date       code                                                        
2019-01-24 000001  0.766334  0.000765  0.012615 -0.000495  25.866452   
           000009  1.356675 -0.000322 -0.002641 -0.001969  22.755220   
           000010  0.799467 -0.001852 -0.007308 -0.000681  21.724602   
           000011  1.223661 -0.000595 -0.012535 -0.002605  22.302695   
           000012  1.052968 -0.000951  0.001668 -0.003550  23.039117   
...                     ...       ...       ...       ...        ...   
2021-12-31 603992  0.164744  0.000370 -0.009153  0.000025  22.780537   
           603993  1.563441 -0.002527  0.001222  0.000496  25.501501   
           603997  1.002980  0.003754 -0.029944  0.001544  23.613248   
           603998  0.801932  0.001239  0.014419  0.000700  21.754922   
           603999  0.902850 -0.000366 -0.001994  0.000290  21.864148   

                         bp  earnings_yield  liquidity    resvol    sizenl  
date       code                                                             
2019-01-24 000001  0.778016       -0.032293   0.574840  0.014329  2.444768  
           000009  1.447453        0.060426  -0.120262 -0.020269 -0.986730  
           000010  6.490713       -0.060146  -0.095654  0.040743  0.184759  
           000011  1.453638        0.043212   0.971464 -0.012230 -0.652478  
           000012  1.113183        0.039006  -0.581585 -0.020129 -1.049638  
...                     ...             ...        ...       ...       ...  
2021-12-31 603992  3.514636        0.009984   0.516717  0.013260 -0.706449  
           603993  2.983824        0.012972   0.216822 -0.016950  1.566283  
           603997  3.866314       -0.002434   0.111446  0.036103 -1.032448  
           603998  2.345817        0.057303  -0.329755  0.005493  0.868782  
           603999  1.682837        0.027646  -1.125015 -0.025946  0.640966  

[1794285 rows x 10 columns]

In [6]:
final.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3247889 entries, (Timestamp('2017-01-03 00:00:00'), '000001') to (Timestamp('2021-12-31 00:00:00'), '605599')
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   beta            float64
 1   alpha           float64
 2   residual        float64
 3   momentum        float64
 4   sizelg          float64
 5   bp              float64
 6   earnings_yield  float64
 7   liquidity       float64
 8   resvol          float64
 9   sizenl          float64
dtypes: float64(10)
memory usage: 260.3+ MB
